In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import json
import shutil
from tqdm import tqdm

In [2]:
# Read data from json file

json_data =  'cropped_data/cropped_annotation/cropped_annotation_mark_added.json' #'annotations/cropped_sampled_seed4.json'
json_object = json.load(open(json_data, "r", encoding='utf-8'))

# For test only 
r = 0
for k,v in json_object.items():
    print(v)
    #print(v['jpg_img_path'])
    r += 1
    if r > 100:
        break

In [3]:
# Sample each image and plot the ground truth
c = 0
img_output_path = '/media/liverusg/4TB/overlayed_cropped_with_mark_image/'

for k,v in json_object.items():
    #fig = plt.figure(figsize=(20,10))
    
   
    # Get dataset type
    data_type = v['set']
    
    zone = data_type.split("_")[0].strip()
    pos_neg = data_type.split("_")[-1].strip()
    '''
    if pos_neg == 'lesions':
        file_path = dataset_path + zone +"/lesions/"
    f pos_neg == 'controls':
        file_path = dataset_path + zone +"/negative_controls/"
    
    file_path += k
    '''
    regions = v['regions']
    file_path  = v['jpg_img_path']
    #print(file_path)
    img = cv2.imread(file_path)
    
    for l in regions:
        if 'structure' in l['region_attributes'].keys():
            lesion_type = l['region_attributes']['structure']
            bbox = l['bbox']
            #print(bbox)
            
            # draw box and text
            cv2.rectangle(img, (bbox['x0'],bbox['y0']), (bbox['x1'], bbox['y1']), (0,0,255), 2) 
            cv2.putText(img, lesion_type, (bbox['x0'],bbox['y0'] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 2) 
            cv2.putText(img, lesion_type, (bbox['x0'],bbox['y0'] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,0), 1) 
            
        elif 'markings' in l['region_attributes'].keys():
            mark = l['region_attributes']['markings']
            mbbox = l['bbox']
            #print(mbbox)
            
            # draw box and text
            cv2.rectangle(img, (mbbox['x0'], mbbox['y0']), (mbbox['x1'], mbbox['y1']), (0,255,0), 2) 
            cv2.putText(img, mark, (mbbox['x0'], mbbox['y0'] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 2) 
            cv2.putText(img, mark, (mbbox['x0'], mbbox['y0'] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,0), 1) 

    #print(img_output_path +data_type+"/"+ k)
    cv2.imwrite(img_output_path +zone+"/"+ k,img)
    #plt.imshow(img)
    #plt.show()
        
    #c +=1
    
    #if c > 1000:
        #break

In [ ]:
def check_oof():
    # defined interest classes
    interest_classes = ['FFC', 'FFS', 'hemangioma', 'HCC', 'cyst']
    
    # Load json file
    json_data = 'cropped_data/cropped_annotation/cropped_annotation.json'
    json_object = json.load(open(json_data, "r", encoding='utf-8'))
    
    for k,v in tqdm(json_object.items()):
        
        data_type = v['set'].split("_")[0].strip()
        file_path  = v['jpg_img_path']
        img = cv2.imread(file_path, 0)
        height, width = img.shape

        # Get all regions of the image
        regions = v['regions']

        for l in regions:
            if 'structure' in l['region_attributes'].keys():
                if l['region_attributes']['structure'] in interest_classes:
                    lesion_type = l['region_attributes']['structure']
                    bbox = l['bbox']

                    x0 = bbox['x0'] 
                    y0 = bbox['y0'] 
                    x1 = bbox['x1'] 
                    y1 = bbox['y1'] 

                    if x0 < 0 or x0 > width or x1 < 0 or x1 > width:
                        print(x0, x1, width)
                        print("Found lesion out of frame (x-value)", k)


                    elif y0 < 0 or y0 > height or y1 < 0 or y1 > height:
                        print(y0,y1,height)
                        print("Found lesion out of frame (y-value)", k)
        

            elif 'markings' in l['region_attributes'].keys():
                mark = l['region_attributes']['markings']
                mbbox = l['bbox']
                
                m_x0 = mbbox['x0'] 
                m_y0 = mbbox['y0'] 
                m_x1 = mbbox['x1'] 
                m_y1 = mbbox['y1'] 

                if m_x0 < 0 or m_x0 > width or m_x1 < 0 or m_x1 > width:
                    print("Found lesion out of frame (x-value)", k)


                elif m_y0 < 0 or m_y0 > height or m_y1 < 0 or m_y1 > height:
                    print("Found lesion out of frame (y-value)", k)
                    
check_oof()

In [ ]:
def crop_data(viz=False, limit=False, save_img=False, write_json=False):
    
    # defined interest classes
    interest_classes = ['FFC', 'FFS', 'hemangioma', 'HCC', 'cyst']
    
    # duplicate original file
    shutil.copy('/home/liverusg/PycharmProjects/AI_USG_liver/experiments/retinanet_mix/annotations/cleaned_sampled_seed4.json','/home/liverusg/PycharmProjects/AI_USG_liver/experiments/retinanet_mix/annotations/cropped_sampled_seed4.json')

    # Load json file
    json_data = 'annotations/cropped_sampled_seed4.json'
    json_object = json.load(open(json_data, "r+", encoding='utf-8'))

    # Start cropped image => Plot bbox => save bounding box into new json files
    img_output_path = 'cropped_data/cropped_images/'
    anno_output_path = 'cropped_data/cropped_annotation/'
    
    # create limit sample
    c = 0
    
    if write_json:
        new_json = open(anno_output_path+'cropped_annotation.json', 'w')
    
    for k,v in tqdm(json_object.items()):
        if viz:
            fig = plt.figure(figsize=(20,10))
        
        file_path  = v['jpg_img_path']
        data_type = v['set'].split("_")[0].strip()
        img = cv2.imread(file_path, 0)
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Get window information
        left_w = int(v['window_min_x'])
        top_w = int(v['window_min_y'])
        right_w = int(v['window_max_x'])
        bot_w = int(v['window_max_y'])

        # Crop image
        crop_img = img[top_w:bot_w, left_w:right_w].copy()
        # Get new image shape
        height, width = crop_img.shape
        
        if write_json:
            v['jpg_img_path'] = img_output_path + data_type+"/"+k
        
        # Save cropped image according to the set of data
        if save_img:
            cv2.imwrite(img_output_path +data_type+"/"+ k, crop_img)

        # Get all regions of the image
        regions = v['regions']

        for l in regions:
            if 'structure' in l['region_attributes'].keys():
                if l['region_attributes']['structure'] in interest_classes:
                    lesion_type = l['region_attributes']['structure']
                    bbox = l['bbox']

                    if viz:
                        # draw box and text
                        cv2.rectangle(img, (bbox['x0'], bbox['y0']), (bbox['x1'], bbox['y1']), (0,0,255), 2) 
                        cv2.putText(img, lesion_type, (bbox['x0'],bbox['y0'] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 2) 
                        cv2.putText(img, lesion_type, (bbox['x0'],bbox['y0'] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,0), 1) 

            elif 'markings' in l['region_attributes'].keys():
                mark = l['region_attributes']['markings']
                mbbox = l['bbox']
                
                if viz:
                    # draw box and text
                    cv2.rectangle(img, (mbbox['x0'], mbbox['y0']), (mbbox['x1'], mbbox['y1']), (0,255,0), 2) 
                    cv2.putText(img, mark, (mbbox['x0'], mbbox['y0'] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 2) 
                    cv2.putText(img, mark, (mbbox['x0'], mbbox['y0'] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,0), 1) 
        
        if viz:
            plt.imshow(img, cmap='gray')
            plt.show()

        #################################################################################################################################

        if viz:
            fig = plt.figure(figsize=(20,10))

        found_shift = False
        count_shift = 0
        
        # For check lesion out of frame
        for l in regions:
            if 'structure' in l['region_attributes'].keys():
                if l['region_attributes']['structure'] in interest_classes:
                    lesion_type = l['region_attributes']['structure']
                    bbox = l['bbox']

                    # draw new_bbox and text
                    new_x0 = bbox['x0'] - left_w
                    new_y0 = bbox['y0'] - top_w
                    new_x1 = bbox['x1'] - left_w
                    new_y1 = bbox['y1'] - top_w

                    if new_x0 < 0 or new_x0 > width or new_x1 < 0 or new_x1 > width:
                        print("Found lesion out of frame (x-value)", k)
                        win_left = left_w - 50 
                        win_right = right_w + 50
                        win_top = top_w - (1 + (50 / width))
                        win_bot = bot_w + (1 + (50 / height))
                        found_shift = True
                        print("Found lesion out of frame")
                        count_shift += 1
                        
                    elif new_y0 < 0 or new_y0 > height or new_y1 < 0 or new_y1 > height:
                        print("Found lesion out of frame (y-value)", k)
                        win_left = left_w - (1 + (50 / height))
                        win_right = right_w + (1 + (50 / height))
                        win_top = top_w - 50
                        win_bot = bot_w + 50
                        found_shift = True
                        print("Found lesion out of frame")
                        count_shift += 1
                else:
                    del l

            elif 'markings' in l['region_attributes'].keys():
                mark = l['region_attributes']['markings']
                mbbox = l['bbox']

                # draw new_mbbox and text
                new_m_x0 = mbbox['x0'] - left_w
                new_m_y0 = mbbox['y0'] - top_w
                new_m_x1 = mbbox['x1'] - left_w
                new_m_y1 = mbbox['y1'] - top_w
                
                if new_m_x0 < 0 or new_m_x0 > width or new_m_x1 < 0 or new_m_x1 > width:
                        print("Found mark out of frame (x-value)", k)
                        win_left = left_w - 50 
                        win_right = right_w + 50
                        win_top = top_w - (1 + (50 / width))
                        win_bot = bot_w + (1 + (50 / height))
                        print("Found lesion out of frame")
                        found_shift = True
                        count_shift += 1
                        
                elif new_m_y0 < 0 or new_m_y0 > height or new_m_y1 < 0 or new_m_y1 > height:
                    print("Found mark out of frame (y-value)", k)
                    win_left = left_w - (1 + (50 / height))
                    win_right = right_w + (1 + (50 / height))
                    win_top = top_w - 50
                    win_bot = bot_w + 50
                    print("Found lesion out of frame")
                    found_shift = True
                    count_shift += 1
                  
        # Recalculate all lesion (found shift)
        if found_shift:
            # Recrop image
            crop_img = img[int(win_top):int(win_bot), int(win_left):int(win_right)].copy()
            if save_img:
                cv2.imwrite(img_output_path +data_type+"/"+ k, crop_img)

            # Get new image shape
            height, width = crop_img.shape
            
            # Recalculate lesion in the images
            for l in regions:
                if 'structure' in l['region_attributes'].keys():
                    if l['region_attributes']['structure'] in interest_classes:
                        lesion_type = l['region_attributes']['structure']
                        bbox = l['bbox']

                        # draw new_bbox and text
                        new_x0 = bbox['x0'] - win_left
                        new_y0 = bbox['y0'] - win_top
                        new_x1 = bbox['x1'] - win_left
                        new_y1 = bbox['y1'] - win_top

                        if write_json:
                            # overwrite lesion bbox
                            l['bbox']['x0'] = new_x0
                            l['bbox']['y0'] = new_y0
                            l['bbox']['x1'] = new_x1
                            l['bbox']['y1'] = new_y1

                        if viz:
                            cv2.rectangle(crop_img, (new_x0, new_y0), (new_x1, new_y1), (255,0,0), 2) 
                            cv2.putText(crop_img, lesion_type, (new_x0, new_y0 - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 2) 
                    else:
                        del l

                elif 'markings' in l['region_attributes'].keys():
                    mark = l['region_attributes']['markings']
                    mbbox = l['bbox']

                    # draw new_mbbox and text
                    new_m_x0 = mbbox['x0'] - win_left
                    new_m_y0 = mbbox['y0'] - win_top
                    new_m_x1 = mbbox['x1'] - win_left
                    new_m_y1 = mbbox['y1'] - win_top

                    if write_json:
                        # overwrite marking box
                        l['bbox']['x0'] = new_m_x0
                        l['bbox']['y0'] = new_m_y0
                        l['bbox']['x1'] = new_m_x1
                        l['bbox']['y1'] = new_m_y1

                    if viz:
                        cv2.rectangle(crop_img, (new_m_x0, new_m_y0), (new_m_x1, new_m_y1), (0,255,255), 2) 
                        cv2.putText(crop_img, mark, (new_m_x0, new_m_y0 - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 2) 
                    
        # In case not shift
        else:
            for l in regions:
                if 'structure' in l['region_attributes'].keys():
                    if l['region_attributes']['structure'] in interest_classes:
                        lesion_type = l['region_attributes']['structure']
                        bbox = l['bbox']

                        # draw new_bbox and text
                        new_x0 = bbox['x0'] - left_w
                        new_y0 = bbox['y0'] - top_w
                        new_x1 = bbox['x1'] - left_w
                        new_y1 = bbox['y1'] - top_w

                        if write_json:
                            # overwrite lesion bbox
                            l['bbox']['x0'] = new_x0
                            l['bbox']['y0'] = new_y0
                            l['bbox']['x1'] = new_x1
                            l['bbox']['y1'] = new_y1

                        if viz:
                            cv2.rectangle(crop_img, (new_x0, new_y0), (new_x1, new_y1), (255,0,0), 2) 
                            cv2.putText(crop_img, lesion_type, (new_x0, new_y0 - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 2) 
                            
                    else:
                        del l

                elif 'markings' in l['region_attributes'].keys():
                    mark = l['region_attributes']['markings']
                    mbbox = l['bbox']

                    # draw new_mbbox and text
                    new_m_x0 = mbbox['x0'] - left_w
                    new_m_y0 = mbbox['y0'] - top_w
                    new_m_x1 = mbbox['x1'] - left_w
                    new_m_y1 = mbbox['y1'] - top_w

                    if write_json:
                        # overwrite marking box
                        l['bbox']['x0'] = new_m_x0
                        l['bbox']['y0'] = new_m_y0
                        l['bbox']['x1'] = new_m_x1
                        l['bbox']['y1'] = new_m_y1

                    if viz:
                        cv2.rectangle(crop_img, (new_m_x0, new_m_y0), (new_m_x1, new_m_y1), (0,255,255), 2) 
                        cv2.putText(crop_img, mark, (new_m_x0, new_m_y0 - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 2) 

        if viz:
            plt.imshow(crop_img, cmap='gray')
            plt.show()
            
        # count file    
        #c+= 1
            
        #print("Finished",c,"files")
        
        if limit:
            if c > 10:
                break
                
    if write_json:
        json.dump(json_object, new_json)

In [ ]:
crop_data(viz=False, limit=False, save_img=True, write_json=True)